# 第9章 时序数据 
2020.06.11   

In [ ]:
# 待补充
# 一 时间类型字段的parser
# 1.使用 read_csv 读数据时, 通过参数 parser_dates 参数解析时间
# 参数说明
#parse_dates : bool or list of int or names or list of lists or dict, default False
#    The behavior is as follows:
#
#    * boolean. If True -> try parsing the index.
#    * list of int or names. e.g. If [1, 2, 3] -> try parsing columns 1, 2, 3
#      each as a separate date column.
#    * list of lists. e.g.  If [[1, 3]] -> combine columns 1 and 3 and parse as
#      a single date column.
#    * dict, e.g. {'foo' : [1, 3]} -> parse columns 1, 3 as date and call
#      result 'foo'
#
#    If a column or index cannot be represented as an array of datetimes,
#    say because of an unparseable value or a mixture of timezones, the column
#    or index will be returned unaltered as an object data type. For
#    non-standard datetime parsing, use ``pd.to_datetime`` after
#    ``pd.read_csv``. To parse an index or column with a mixture of timezones,
#    specify ``date_parser`` to be a partially-applied
#    :func:`pandas.to_datetime` with ``utc=True``. See
#    :ref:`io.csv.mixed_timezones` for more.


In [2]:
import pandas as pd
import numpy as np

## 一、时序的创建
###  1. 四类时间变量
#### 现在理解可能关于③和④有些困惑，后面会作出一些说明

名称 | 描述 | 元素类型 | 创建方式  
:-|:-|:-|:-
① Date times（时间点/时刻） | 描述特定日期或时间点 | Timestamp | to_datetime或date_range
② Time spans（时间段/时期） | 由时间点定义的一段时期 | Period | Period或period_range
③ Date offsets（相对时间差） | 一段时间的相对大小（与夏/冬令时无关） | DateOffset | DateOffset
④ Time deltas（绝对时间差） | 一段时间的绝对大小（与夏/冬令时有关） | Timedelta | to_timedelta或timedelta_range

### 2. 时间点的创建

#### （a）to_datetime方法
#### Pandas在时间点建立的输入格式规定上给了很大的自由度，下面的语句都能正确建立同一时间点

In [112]:
#pd.to_datetime('2020.01.01')   # YYYY.MM.DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.
#pd.to_datetime('2020-12-31')   # YYYY-MM-DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.
#pd.to_datetime('2020 02 01')   # YYYY MM DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.
#pd.to_datetime('2020 01-31')   # YYYY MM-DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.
#pd.to_datetime('2020-05 31')   # YYYY-MM DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.
#pd.to_datetime('2020/01/31')   # YYYY/MM/DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.
#pd.to_datetime('31.1.2020')   # DD.MM.YYYY 格式的日期可以被成功地转换,
#pd.to_datetime('12.31.2020')   # MM.DD.YYYY 格式的日期可以被成功地转换,如DD和MM有一个大于12,会自动推断该值为DD,另一小于12的值为MM
#pd.to_datetime('11.01.2020')   #但'11.01.2020'会被优先默认视作 DD.MM.YYYY格式,月和日前的 0 可有可无.
#pd.to_datetime('11.01.2020',format="%d.%m.%Y") # 除非显式地指定了时间格式参数format
#pd.to_datetime('01 11 2020')    # 会根据前两个字段与12的大小关系自动推断:当均小于12时,优先视作 MM.DD;
#pd.to_datetime('21 11 2020')                                #如果第一个字段大于12,则推断为日期
#pd.to_datetime('21 11-2020')   # 同 pd.to_datetime('11.01.2020')  的情况,月和日前的 0 可有可无.
#pd.to_datetime('1-1 2020')     # 同 pd.to_datetime('11.01.2020')  的情况,月和日前的 0 可有可无.
#pd.to_datetime('1-1-2020')     # 同 pd.to_datetime('11.01.2020')  的情况,月和日前的 0 可有可无.
#pd.to_datetime('31/1/2020')    # 同 pd.to_datetime('11.01.2020')  的情况,月和日前的 0 可有可无.
#pd.to_datetime('20200101')   # YYYY.MM.DD 格式的日期可以被正确地转换,月和日前的 0 可有可无.

#################################
#pd.to_datetime('2020 12.31')    # YYYY MM.DD 格式的日期未被正确转换,而是转换为了该月的第一天. 
#pd.to_datetime('1.11 2020')      #  M.DD YYYY 格式的日期未被正确转换,而是转换为了该月的第一天
#pd.to_datetime('01.11 2020')      # MM.DD YYYY 格式的日期则可以被正确转换,这种情况下, 月份前的 0 不能省略
#pd.to_datetime('11.2 2020')      #  MM.D YYYY 格式的日期未被正确转换,而是转换为了该月的第一天
#pd.to_datetime('11.2 2020',format='%m.%d %Y')      #  指定了 format 参数后,MM.D YYYY 格式的日期可以被正确转换
#pd.to_datetime('11.02 2020')      #  MM.DD YYYY 格式的日期则可以被正确转换,这种情况下, 月份前的 0 不能省略
#pd.to_datetime('2020.0111')      #  YYYY.MMDD 格式的日期未被正确转换,而是转换为了该月的第一天.


In [106]:
pd.to_datetime??

Signature:
pd.to_datetime(
    arg,
    errors='raise',
    dayfirst=False,
    yearfirst=False,
    utc=None,
    format=None,
    exact=True,
    unit=None,
    infer_datetime_format=False,
    origin='unix',
    cache=True,
)
Source:   
def to_datetime(
    arg,
    errors="raise",
    dayfirst=False,
    yearfirst=False,
    utc=None,
    format=None,
    exact=True,
    unit=None,
    infer_datetime_format=False,
    origin="unix",
    cache=True,
):
    """
    Convert argument to datetime.

    Parameters
    ----------
    arg : int, float, str, datetime, list, tuple, 1-d array, Series DataFrame/dict-like
        The object to convert to a datetime.
    errors : {'ignore', 'raise', 'coerce'}, default 'raise'
        - If 'raise', then invalid parsing will raise an exception.
        - If 'coerce', then invalid parsing will be set as NaT.
        - If 'ignore', then invalid parsing will return the input.
    dayfirst : bool, default False
        Specify a date parse order if `a

#### 下面的语句都会报错

In [131]:
#pd.to_datetime('2020\\1\\1') # windows 系统下并没有报错
#pd.to_datetime('2020`1`1')
#pd.to_datetime('2020.1 1') # ParserError: Unknown string format: 2020.1 1
#pd.to_datetime('2020.01 1') # 但在.后的数字前加 0 凑够两位就不会报错了
#pd.to_datetime('2020.01 11') # 这时候是把2020.01 中的 01视作DD了.
#pd.to_datetime('1 1.2020')  # ParserError: Unknown string format: 1 1.2020
#pd.to_datetime('01 12.2020') #但凑够位数就不会报错了, 且 .2020 前的数字被默认识别为 MM DD
#pd.to_datetime('31 12.2020') #但如果第一个数字大于12,则会被识别为MM DD.YYYY

Timestamp('2020-12-31 00:00:00')

#### 此时可利用format参数强制匹配

In [139]:
# 最稳妥的办法是使用format参数指定日期格式,避免出现各种混淆和错误.
#pd.to_datetime('2020\\1\\1',format='%Y\\%m\\%d')
#pd.to_datetime('2020`1`1',format='%Y`%m`%d')
#pd.to_datetime('2020.1 1',format='%Y.%m %d')
print(pd.to_datetime('1 1.2020',format='%d %m.%Y')) # 使用print函数可以格式化输出结果

2020-01-01 00:00:00


#### 同时，为pd.to_datetime传入列表可以将其转为时间点索引(DatetimeIndex 类型)

In [140]:
pd.Series(range(2),index=pd.to_datetime(['2020/1/1','2020/1/2']))

2020-01-01    0
2020-01-02    1
dtype: int64

In [142]:
print(pd.to_datetime(['2020/1/1','2020/1/2']))

DatetimeIndex(['2020-01-01', '2020-01-02'], dtype='datetime64[ns]', freq=None)


In [143]:
type(pd.to_datetime(['2020/1/1','2020/1/2']))

pandas.core.indexes.datetimes.DatetimeIndex

#### 对于DataFrame而言，如果列已经按照时间顺序排好，则利用to_datetime可自动转换

In [144]:
df = pd.DataFrame({'year': [2020, 2020],'month': [1, 1], 'day': [1, 2]})
df

,year,month,day
0,2020,1,1
1,2020,1,2


In [145]:
pd.to_datetime(df)

0   2020-01-01
1   2020-01-02
dtype: datetime64[ns]

In [146]:
type(pd.to_datetime(df)) # 作用后的结果是个Series

pandas.core.series.Series

#### （b）时间精度与范围限制
#### 事实上，Timestamp的精度远远不止day，可以最小到纳秒ns

In [147]:
pd.to_datetime('2020/1/1 00:00:00.123456789')

Timestamp('2020-01-01 00:00:00.123456789')

#### 同时，它带来范围的代价就是只有大约584年的时间点是可用的

In [154]:
print(pd.Timestamp.min,pd.Timestamp.max)

1677-09-21 00:12:43.145225 2262-04-11 23:47:16.854775807


In [155]:
# 相对而言,datetime 的表示范围要大得多
print(pd.datetime.min,pd.datetime.max)
# FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

0001-01-01 00:00:00 9999-12-31 23:59:59.999999


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


#### （c）date_range方法
#### 一般来说，start/end/periods（时间点个数）/freq（间隔方法）是该方法最重要的参数，给定了其中的3个，剩下的一个就会被确定

In [156]:
pd.date_range(start='2020/1/1',end='2020/1/10',periods=3) # 指定生成的时间点个数
# periods : int, optional    Number of periods to generate.

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-05 12:00:00',
               '2020-01-10 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [157]:
pd.date_range??

Signature:
pd.date_range(
    start=None,
    end=None,
    periods=None,
    freq=None,
    tz=None,
    normalize=False,
    name=None,
    closed=None,
    **kwargs,
) -> pandas.core.indexes.datetimes.DatetimeIndex
Source:   
def date_range(
    start=None,
    end=None,
    periods=None,
    freq=None,
    tz=None,
    normalize=False,
    name=None,
    closed=None,
    **kwargs,
) -> DatetimeIndex:
    """
    Return a fixed frequency DatetimeIndex.

    Parameters
    ----------
    start : str or datetime-like, optional
        Left bound for generating dates.
    end : str or datetime-like, optional
        Right bound for generating dates.
    periods : int, optional
        Number of periods to generate.
    freq : str or DateOffset, default 'D'
        Frequency strings can have multiples, e.g. '5H'. See
        :ref:`here <timeseries.offset_aliases>` for a list of
        frequency aliases.
    tz : str or tzinfo, optional
        Time zone name for returning localized Dat

In [209]:
pd.date_range(start='2020/1/1',end='2020/1/10',freq='D') # 指定生成的时间间隔长度,'D' 是默认的,表示天
# freq : str or DateOffset, default 'D'
#        Frequency strings can have multiples, e.g. '5H'. See:  ref:`here <timeseries.offset_aliases>` for a list of frequency aliases.

DatetimeIndex(['2020-01-01'], dtype='datetime64[ns]', freq='BMS')

In [210]:
# 指定时间起点,和时间点个数,时间间隔长度, 来生成时间点序列
pd.date_range(start='2020/1/1',periods=6,freq='BMS') 
# freq参数'S'或's'表示秒, 'T'或't','min'表示分钟, 'H'或'h'表示小时, 'W'或'w'表示周,使用周初(从周日算起)
#         'D'表示日历日, 'B'表示工作日(不包括周末),'C'=custom business day frequency 
#         'M'或'm'表示月,会自动转换为月尾, 'MS'表示月初,'BMS'=business month start frequency
#         'Q'或'q'表示季度,会自动转换为季度最后一天, 'QS'表示季度第一天
#         'Y'表示年,使用年尾('A-DEC'), 'YS'或'AS'表示年,使用年初('AS-JAN')--修改为'AS-OCT'则使用10月的月初

DatetimeIndex(['2020-01-01', '2020-02-03', '2020-03-02', '2020-04-01',
               '2020-05-01', '2020-06-01'],
              dtype='datetime64[ns]', freq='BMS')

In [208]:
# 指定时间终点, 和时间点的个数,时间间隔的长度,来生成时间点序列
pd.date_range(end='2020/1/3',periods=3,freq='W') #freq参数同上

DatetimeIndex(['2019-12-15', '2019-12-22', '2019-12-29'], dtype='datetime64[ns]', freq='W-SUN')

#### 其中freq参数有许多选项，下面将常用部分罗列如下
[表格对齐存在问题]

符号 | D/B | W | M/Q/Y | BM/BQ/BY | MS/QS/YS | BMS/BQS/BYS | H | T | S
:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
描述 | 日/工作日 | 周 | 月末 | 月/季/年末日 | 月/季/年末工作日 | 月/季/年初日 | 月/季/年初工作日 | 小时 | 分钟 |秒

更多选项可看[这里](https://pandas.pydata.org/docs/user_guide/timeseries.html#offset-aliases)

In [211]:
pd.date_range(start='2020/1/1',periods=3,freq='T')

DatetimeIndex(['2020-01-01 00:00:00', '2020-01-01 00:01:00',
               '2020-01-01 00:02:00'],
              dtype='datetime64[ns]', freq='T')

In [212]:
pd.date_range(start='2020/1/1',periods=3,freq='M')

DatetimeIndex(['2020-01-31', '2020-02-29', '2020-03-31'], dtype='datetime64[ns]', freq='M')

In [213]:
pd.date_range(start='2020/1/1',periods=3,freq='BYS')

DatetimeIndex(['2020-01-01', '2021-01-01', '2022-01-03'], dtype='datetime64[ns]', freq='BAS-JAN')

#### bdate_range是一个类似与date_range的方法，特点在于可以在自带的工作日间隔设置上，再选择weekmask参数和holidays参数
#### 它的freq中有一个特殊的'C'/'CBM'/'CBMS'选项，表示定制，需要联合weekmask参数和holidays参数使用
#### 例如现在需要将工作日中的周一、周二、周五3天保留，并将部分holidays剔除

In [18]:
weekmask = 'Mon Tue Fri'
holidays = [pd.Timestamp('2020/1/%s'%i) for i in range(7,13)]
#注意holidays
pd.bdate_range(start='2020-1-1',end='2020-1-15',freq='C',weekmask=weekmask,holidays=holidays)

DatetimeIndex(['2020-01-03', '2020-01-06', '2020-01-13', '2020-01-14'], dtype='datetime64[ns]', freq='C')

### 3. DateOffset对象

#### （a）DataOffset与Timedelta的区别
#### Timedelta绝对时间差的特点指无论是冬令时还是夏令时，增减1day都只计算24小时
#### DataOffset相对时间差指，无论一天是23\24\25小时，增减1day都与当天相同的时间保持一致
#### 例如，英国当地时间 2020年03月29日，01:00:00 时钟向前调整 1 小时 变为 2020年03月29日，02:00:00，开始夏令时

In [214]:
ts = pd.Timestamp('2020-3-29 01:00:00', tz='Europe/Helsinki')
ts + pd.Timedelta(days=1)

Timestamp('2020-03-30 02:00:00+0300', tz='Europe/Helsinki')

In [215]:
ts + pd.DateOffset(days=1)

Timestamp('2020-03-30 01:00:00+0300', tz='Europe/Helsinki')

#### 这似乎有些令人头大，但只要把tz（time zone）去除就可以不用管它了，两者保持一致，除非要使用到时区变换

In [217]:
ts = pd.Timestamp('2020-3-29 01:00:00')
ts

Timestamp('2020-03-29 01:00:00')

In [221]:
pd.Timedelta(days=1)

Timedelta('1 days 00:00:00')

In [222]:
ts + pd.Timedelta(days=1)

Timestamp('2020-03-30 01:00:00')

In [223]:
ts + pd.DateOffset(days=1)

Timestamp('2020-03-30 01:00:00')

In [220]:
pd.DateOffset??
# 基本的数据类型
#        - year
#        - month
#        - day
#        - weekday
#        - hour
#        - minute
#        - second
#        - microsecond
#        - nanosecond

Init signature: pd.DateOffset(n=1, normalize=False, **kwds)
Source:        
class DateOffset(BaseOffset):
    """
    Standard kind of date increment used for a date range.

    Works exactly like relativedelta in terms of the keyword args you
    pass in, use of the keyword n is discouraged-- you would be better
    off specifying n in the keywords you use, but regardless it is
    there for you. n is needed for DateOffset subclasses.

    DateOffset work as follows.  Each offset specify a set of dates
    that conform to the DateOffset.  For example, Bday defines this
    set to be the set of dates that are weekdays (M-F).  To test if a
    date is in the set of a DateOffset dateOffset we can use the
    is_on_offset method: dateOffset.is_on_offset(date).

    If a date is not on a valid date, the rollback and rollforward
    methods can be used to roll the date to the nearest valid date
    before/after the date.

    DateOffsets can be created to move dates forward a given number o

In [308]:
#  Timedelta 各类型的时间长度
print(f"1Y = 1y = {pd.Timedelta('1y')}\n1DAY(S) = 1day(s) = 1D = 1d = {pd.Timedelta('1day')}\n1W = 1w = {pd.Timedelta('1w')}\n\
1M = 1m = {pd.Timedelta('1M')}\n1T = 1t = {pd.Timedelta('1t')}\n1S = 1s = {pd.Timedelta('1s')}\n\
1L = 1l = {pd.Timedelta('1l')}\n1U = 1u = {pd.Timedelta('1u')}\n1N = 1n = {pd.Timedelta('1n')}, 1000N = {pd.Timedelta('1n')*1000}\n")

1Y = 1y = 365 days 05:49:12
1DAY(S) = 1day(s) = 1D = 1d = 1 days 00:00:00
1W = 1w = 7 days 00:00:00
1M = 1m = 0 days 00:01:00
1T = 1t = 0 days 00:01:00
1S = 1s = 0 days 00:00:01
1L = 1l = 0 days 00:00:00.001000
1U = 1u = 0 days 00:00:00.000001
1N = 1n = 0 days 00:00:00.000000, 1000N = 0 days 00:00:00.000001



In [317]:
# Timedelta 各类型之间的运算
# 天 与 小时,分钟,秒,微秒,纳秒的加法
#pd.Timedelta(days=1) + pd.Timedelta(hours=1)         # Timedelta('1 days 01:00:00')
#pd.Timedelta(days=1) + pd.Timedelta(minutes=1)       # Timedelta('1 days 00:01:00')
#pd.Timedelta(days=1) + pd.Timedelta(seconds=1)       # Timedelta('1 days 00:00:01')
#pd.Timedelta(days=1) + pd.Timedelta(microseconds=1)  # Timedelta('1 days 00:00:00.000001')
#pd.Timedelta(days=1) + pd.Timedelta(nanoseconds=1)   # Timedelta('1 days 00:00:00.000000')

#pd.Timedelta(hours=1) + pd.Timedelta(minutes=1)      # Timedelta('0 days 01:01:00')
#pd.Timedelta(hours=1) + pd.Timedelta(seconds=1)      # Timedelta('0 days 01:00:01')
#pd.Timedelta(hours=1) + pd.Timedelta(microseconds=1) # Timedelta('0 days 01:00:00.000001')
#pd.Timedelta(hours=1) + pd.Timedelta(nanoseconds=1)  # Timedelta('0 days 01:00:00.000000')


In [318]:
pd.Timedelta('1n')*1000

Timedelta('0 days 00:00:00.000001')

In [319]:
pd.Timedelta??

Init signature:
pd.Timedelta(
    value=<object object at 0x00000000050A8960>,
    unit=None,
    **kwargs,
)
Docstring:     
Represents a duration, the difference between two dates or times.

Timedelta is the pandas equivalent of python's ``datetime.timedelta``
and is interchangeable with it in most cases.

Parameters
----------
value : Timedelta, timedelta, np.timedelta64, string, or integer
unit : str, default 'ns'
    Denote the unit of the input, if input is an integer.

    Possible values:

    * 'Y', 'M', 'W', 'D', 'T', 'S', 'L', 'U', or 'N'
    * 'days' or 'day'
    * 'hours', 'hour', 'hr', or 'h'
    * 'minutes', 'minute', 'min', or 'm'
    * 'seconds', 'second', or 'sec'
    * 'milliseconds', 'millisecond', 'millis', or 'milli'
    * 'microseconds', 'microsecond', 'micros', or 'micro'
    * 'nanoseconds', 'nanosecond', 'nanos', 'nano', or 'ns'.

**kwargs
    Available kwargs: {days, seconds, microseconds,
    milliseconds, minutes, hours, weeks}.
    Values for construction 

* datetime 库中的模块和方法  
待补充    
* [numpy中的与时间相关的数据类型和方法](https://numpy.org.cn/reference/arrays/datetime.html#%E5%9F%BA%E6%9C%AC%E6%97%A5%E6%9C%9F%E6%97%B6%E9%97%B4)

In [309]:
import datetime

In [310]:
print(dir(datetime))

['MAXYEAR', 'MINYEAR', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'date', 'datetime', 'datetime_CAPI', 'sys', 'time', 'timedelta', 'timezone', 'tzinfo']


In [311]:
print(dir(datetime.datetime))

['__add__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__rsub__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', 'astimezone', 'combine', 'ctime', 'date', 'day', 'dst', 'fold', 'fromisoformat', 'fromordinal', 'fromtimestamp', 'hour', 'isocalendar', 'isoformat', 'isoweekday', 'max', 'microsecond', 'min', 'minute', 'month', 'now', 'replace', 'resolution', 'second', 'strftime', 'strptime', 'time', 'timestamp', 'timetuple', 'timetz', 'today', 'toordinal', 'tzinfo', 'tzname', 'utcfromtimestamp', 'utcnow', 'utcoffset', 'utctimetuple', 'weekday', 'year']


In [312]:
print(dir(datetime.timedelta))

['__abs__', '__add__', '__bool__', '__class__', '__delattr__', '__dir__', '__divmod__', '__doc__', '__eq__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__mod__', '__mul__', '__ne__', '__neg__', '__new__', '__pos__', '__radd__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmod__', '__rmul__', '__rsub__', '__rtruediv__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', 'days', 'max', 'microseconds', 'min', 'resolution', 'seconds', 'total_seconds']


In [316]:
print(dir(datetime.sys))

['__breakpointhook__', '__displayhook__', '__doc__', '__excepthook__', '__interactivehook__', '__loader__', '__name__', '__package__', '__spec__', '__stderr__', '__stdin__', '__stdout__', '_base_executable', '_clear_type_cache', '_current_frames', '_debugmallocstats', '_enablelegacywindowsfsencoding', '_framework', '_getframe', '_git', '_home', '_xoptions', 'api_version', 'argv', 'base_exec_prefix', 'base_prefix', 'breakpointhook', 'builtin_module_names', 'byteorder', 'call_tracing', 'callstats', 'copyright', 'displayhook', 'dllhandle', 'dont_write_bytecode', 'exc_info', 'excepthook', 'exec_prefix', 'executable', 'exit', 'flags', 'float_info', 'float_repr_style', 'get_asyncgen_hooks', 'get_coroutine_origin_tracking_depth', 'get_coroutine_wrapper', 'getallocatedblocks', 'getcheckinterval', 'getdefaultencoding', 'getfilesystemencodeerrors', 'getfilesystemencoding', 'getprofile', 'getrecursionlimit', 'getrefcount', 'getsizeof', 'getswitchinterval', 'gettrace', 'getwindowsversion', 'hash_i

#### （b）增减一段时间
#### DateOffset的可选参数包括years/months/weeks/days/hours/minutes/seconds

In [320]:
# DateOffset 的数据类型以及它们之间的运算
pd.Timestamp('2020-01-01') + pd.DateOffset(minutes=20) - pd.DateOffset(weeks=2)

Timestamp('2019-12-18 00:20:00')

#### （c）各类常用offset对象

freq | D/B | W | (B)M/(B)Q/(B)Y | (B)MS/(B)QS/(B)YS | H | T | S | C |
:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:
offset | DateOffset/BDay | Week | (B)MonthEnd/(B)QuarterEnd/(B)YearEnd | (B)MonthBegin/(B)QuarterBegin/(B)YearBegin | Hour | Minute | Second | CDay(定制工作日)

In [321]:
pd.Timestamp('2020-01-01') + pd.offsets.Week(2)

Timestamp('2020-01-15 00:00:00')

In [322]:
pd.Timestamp('2020-01-01') + pd.offsets.BQuarterBegin(1)

Timestamp('2020-03-02 00:00:00')

#### （d）序列的offset操作
#### 利用apply函数

In [324]:
pd.Series(pd.offsets.BYearBegin(3).apply(i) for i in pd.date_range('20200101',periods=3,freq='Y'))

0   2023-01-02
1   2024-01-01
2   2025-01-01
dtype: datetime64[ns]

In [325]:
pd.offsets.BYearBegin(3)

<3 * BusinessYearBegins: month=1>

In [326]:
pd.date_range('20200101',periods=3,freq='Y')

DatetimeIndex(['2020-12-31', '2021-12-31', '2022-12-31'], dtype='datetime64[ns]', freq='A-DEC')

In [328]:
pd.date_range('20200101',periods=3,freq='Y')[0]

Timestamp('2020-12-31 00:00:00', freq='A-DEC')

In [329]:
pd.offsets.BYearBegin(3).apply(pd.date_range('20200101',periods=3,freq='Y')[0])
# apply方法相当于把 DateOffset 对象 <3 * BusinessYearBegins: month=1> 加到 传入的 Timestamp 类型的参数上

Timestamp('2023-01-02 00:00:00', freq='A-DEC')

In [332]:
BY_3 = pd.offsets.BYearBegin(3)
BY_3.apply??

Signature: BY_3.apply(other)
Docstring: <no docstring>
Source:   
    @apply_wraps
    def apply(self, other):
        years = roll_yearday(other, self.n, self.month, self._day_opt)
        months = years * 12 + (self.month - other.month)
        return shift_month(other, months, self._day_opt)
File:      c:\programdata\anaconda3\lib\site-packages\pandas\tseries\offsets.py
Type:      method


#### 直接使用对象加减

In [333]:
pd.date_range('20200101',periods=3,freq='Y') + pd.offsets.BYearBegin(3)

DatetimeIndex(['2023-01-02', '2024-01-01', '2025-01-01'], dtype='datetime64[ns]', freq='A-DEC')

#### 定制offset，可以指定weekmask和holidays参数（思考为什么三个都是一个值）

In [340]:
pd.Series(pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020060').apply(i)
                                  for i in pd.date_range('20200605',periods=3,freq='D'))

0   2020-06-17
1   2020-06-17
2   2020-06-17
dtype: datetime64[ns]

In [341]:
pd.date_range('20200605',periods=3,freq='D')

DatetimeIndex(['2020-06-05', '2020-06-06', '2020-06-07'], dtype='datetime64[ns]', freq='D')

In [342]:
pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020060')
# 周掩码值用星期单词缩写列出所有的工作日, 本例中, 只有周三和周五是工作日

<3 * CustomBusinessDays>

In [347]:
pd.offsets.CDay(3,weekmask='Wed Fri',holidays='2020060').apply(pd.date_range('20200605',periods=3,freq='D')[0])
# 20200605是周五,从这天往后数三个工作日,就得到下下周的周三,即 20200617

Timestamp('2020-06-17 00:00:00')

In [350]:
# 如果将data_range改为从20200604(周四开始)开始, 并把向后数的工作日改为 4 个 
# 则由于 20200605是工作日, 20200604后的四个工作日为20200617,但20200605,20200606后的四个工作日是20200619
print(pd.date_range('20200604',periods=3,freq='D'))
pd.Series(pd.offsets.CDay(4,weekmask='Wed Fri').apply(i)
                                  for i in pd.date_range('20200604',periods=3,freq='D'))

DatetimeIndex(['2020-06-04', '2020-06-05', '2020-06-06'], dtype='datetime64[ns]', freq='D')


0   2020-06-17
1   2020-06-19
2   2020-06-19
dtype: datetime64[ns]

In [343]:
pd.offsets.CDay??

Init signature:
pd.offsets.CDay(
    n=1,
    normalize=False,
    weekmask='Mon Tue Wed Thu Fri',
    holidays=None,
    calendar=None,
    offset=datetime.timedelta(0),
)
Source:        
class CustomBusinessDay(_CustomMixin, BusinessDay):
    """
    DateOffset subclass representing possibly n custom business days,
    excluding holidays.

    Parameters
    ----------
    n : int, default 1
    normalize : bool, default False
        Normalize start/end dates to midnight before generating date range.
    weekmask : str, Default 'Mon Tue Wed Thu Fri'
        Weekmask of valid business days, passed to ``numpy.busdaycalendar``.
    holidays : list
        List/array of dates to exclude from the set of valid business days,
        passed to ``numpy.busdaycalendar``.
    calendar : pd.HolidayCalendar or np.busdaycalendar
    offset : timedelta, default timedelta(0)
    """

    _prefix = "C"
    _attributes = frozenset(
        ["n", "normalize", "weekmask", "holidays", "calendar", "offs

## 二、时序的索引及属性
### 1. 索引切片
#### 这一部分几乎与第二章的规则完全一致

In [351]:
rng = pd.date_range(start = '2020',end = '2021', freq='W')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.head()

2020-01-05   -0.193085
2020-01-12   -2.148704
2020-01-19   -0.100871
2020-01-26    0.915887
2020-02-02    0.364951
Freq: W-SUN, dtype: float64

In [352]:
ts['2020-01-26']

0.9158871837808515

#### 合法字符自动转换为时间点

In [358]:
#ts['2020-01-26':'20200726'].head()
#ts['2020-01-26':'2020.07.26'].head()
#ts['2020-01-26':'2020 07.26'].head()
#ts['2020-01-26':'2020 07 26'].head()
#ts['2020-01-26':'2020-07 26'].head()
#ts['2020-01-26':'2020 07-26'].head()
# 关于"合法字符"可参考本章开头的 pd.to_datetime 

2020-01-26    0.915887
2020-02-02    0.364951
2020-02-09    1.135340
2020-02-16    1.106814
2020-02-23   -0.235867
Freq: W-SUN, dtype: float64

### 2. 子集索引

In [359]:
ts['2020-7'].head()

2020-07-05   -0.110543
2020-07-12    0.727864
2020-07-19    1.484013
2020-07-26    1.331541
Freq: W-SUN, dtype: float64

In [361]:
# 子集也可以使用区间切片的方式
ts['2020-7':'2020-9'].head(10)

2020-07-05   -0.110543
2020-07-12    0.727864
2020-07-19    1.484013
2020-07-26    1.331541
2020-08-02    0.104710
2020-08-09    0.813628
2020-08-16   -0.518019
2020-08-23    0.198180
2020-08-30    0.847798
2020-09-06   -0.320273
Freq: W-SUN, dtype: float64

#### 支持混合形态索引

In [365]:
#ts['2011-1':'20200726'].head()
#ts['2011-1':'2020 07 26'].head()
#ts['2011-1':'2020-07 26'].head()
ts['2011-1':'2020 07-26'].head()
# 只要是合法格式, 都可以用来切片

2020-01-05   -0.193085
2020-01-12   -2.148704
2020-01-19   -0.100871
2020-01-26    0.915887
2020-02-02    0.364951
Freq: W-SUN, dtype: float64

### 3. 时间点的属性

#### 采用dt对象可以轻松获得关于时间的信息

In [366]:
pd.Series(ts.index).dt.week.head()

0    1
1    2
2    3
3    4
4    5
dtype: int64

In [367]:
pd.Series(ts.index).dt.day.head()

0     5
1    12
2    19
3    26
4     2
dtype: int64

In [398]:
print(dir(pd.Series(ts.index).dt))

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_accessors', '_add_delegate_accessors', '_constructor', '_delegate_method', '_delegate_property_get', '_delegate_property_set', '_deprecations', '_dir_additions', '_dir_deletions', '_freeze', '_get_values', '_reset_cache', 'ceil', 'date', 'day', 'day_name', 'dayofweek', 'dayofyear', 'days_in_month', 'daysinmonth', 'floor', 'freq', 'hour', 'is_leap_year', 'is_month_end', 'is_month_start', 'is_quarter_end', 'is_quarter_start', 'is_year_end', 'is_year_start', 'microsecond', 'minute', 'month', 'month_name', 'nanosecond', 'normalize', 'quarter', 'round', 'second', 'strftime', 'time', 'timetz', 'to_period', 'to_pydatetime', 'tz', 'tz

In [400]:
type(pd.Series(ts.index).dt.year)
# 怎样筛选 dir(pd.Series(ts.index).dt) 中类型为 Series 的?

pandas.core.series.Series

In [401]:
type(pd.Series(ts.index).dt.date)

pandas.core.series.Series

In [402]:
type(pd.Series(ts.index).dt.day)

pandas.core.series.Series

#### 利用strftime可重新修改时间格式

In [409]:
pd.Series(ts.index).dt.strftime('%Y-间隔1-%m-间隔2-%d').head()

0    2020-01-05 00:00:00
1    2020-01-12 00:00:00
2    2020-01-19 00:00:00
3    2020-01-26 00:00:00
4    2020-02-02 00:00:00
dtype: object

In [407]:
pd.Series(ts.index).head() # dtype 为 datetime64[ns] 

0   2020-01-05
1   2020-01-12
2   2020-01-19
3   2020-01-26
4   2020-02-02
dtype: datetime64[ns]

In [416]:
ts.index

DatetimeIndex(['2020-01-05', '2020-01-12', '2020-01-19', '2020-01-26',
               '2020-02-02', '2020-02-09', '2020-02-16', '2020-02-23',
               '2020-03-01', '2020-03-08', '2020-03-15', '2020-03-22',
               '2020-03-29', '2020-04-05', '2020-04-12', '2020-04-19',
               '2020-04-26', '2020-05-03', '2020-05-10', '2020-05-17',
               '2020-05-24', '2020-05-31', '2020-06-07', '2020-06-14',
               '2020-06-21', '2020-06-28', '2020-07-05', '2020-07-12',
               '2020-07-19', '2020-07-26', '2020-08-02', '2020-08-09',
               '2020-08-16', '2020-08-23', '2020-08-30', '2020-09-06',
               '2020-09-13', '2020-09-20', '2020-09-27', '2020-10-04',
               '2020-10-11', '2020-10-18', '2020-10-25', '2020-11-01',
               '2020-11-08', '2020-11-15', '2020-11-22', '2020-11-29',
               '2020-12-06', '2020-12-13', '2020-12-20', '2020-12-27'],
              dtype='datetime64[ns]', freq='W-SUN')

In [419]:
ts.index.to_frame().reset_index().head()

,index,0
0,2020-01-05,2020-01-05
1,2020-01-12,2020-01-12
2,2020-01-19,2020-01-19
3,2020-01-26,2020-01-26
4,2020-02-02,2020-02-02


In [422]:
ts.index.to_frame().reset_index()['index'].head()
# 此时还是  datetime64[ns]

0   2020-01-05
1   2020-01-12
2   2020-01-19
3   2020-01-26
4   2020-02-02
Name: index, dtype: datetime64[ns]

In [421]:
# 但是使用 dt.strftime 提取了日期后, 就成了 object 了
ts.index.to_frame().reset_index()['index'].dt.strftime('%Y.%m.%d').head()
# 这是由于 该函数的返回值是字符串
# Returns
# -------
# ndarray
#     NumPy ndarray of formatted strings.

0    2020.01.05
1    2020.01.12
2    2020.01.19
3    2020.01.26
4    2020.02.02
Name: index, dtype: object

In [423]:
# 如果还想保持为  datetime64[ns] ,需要转换一次
ts.index.to_frame().reset_index()['index'].dt.strftime('%Y.%m.%d').astype('datetime64[ns]').head()

0   2020-01-05
1   2020-01-12
2   2020-01-19
3   2020-01-26
4   2020-02-02
Name: index, dtype: datetime64[ns]

In [426]:
tmp = pd.Series(ts.index)
tmp.dt.strftime??

Signature: tmp.dt.strftime(*args, **kwargs)
Docstring:
Convert to Index using specified date_format.

Return an Index of formatted strings specified by date_format, which
supports the same string format as the python standard library. Details
of the string format can be found in `python string format
doc <https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior>`__.

Parameters
----------
date_format : str
    Date format string (e.g. "%Y-%m-%d").

Returns
-------
ndarray
    NumPy ndarray of formatted strings.

See Also
--------
to_datetime : Convert the given argument to datetime.
DatetimeIndex.normalize : Return DatetimeIndex with times to midnight.
DatetimeIndex.round : Round the DatetimeIndex to the specified freq.
DatetimeIndex.floor : Floor the DatetimeIndex to the specified freq.

Examples
--------
>>> rng = pd.date_range(pd.Timestamp("2018-03-10 09:00"),
...                     periods=3, freq='s')
>>> rng.strftime('%B %d, %Y, %r')
Index(['March 10, 2018,

In [428]:
# 需要进一步了解关于datetime的知识
# https://blog.csdn.net/cmzsteven/article/details/64906245

#### 对于datetime对象可以直接通过属性获取信息

In [432]:
pd.date_range('2020','2021', freq='W').month

Int64Index([ 1,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,  4,
             5,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  8,  8,  8,  8,
             8,  9,  9,  9,  9, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12,
            12],
           dtype='int64')

In [440]:
pd.date_range('2020','2021', freq='W').weekday # 周几,一周中的第几天,6表示第七天--使用的是 ISO 标准周,一个星期是从星期一开始算起的???

Int64Index([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
            6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
            6, 6, 6, 6, 6, 6, 6, 6],
           dtype='int64')

In [445]:
# 例如, 2020-06-14 是星期日, 是一周中的第七天, 则对应的 weekday = 6
pd.date_range('2020','2021', freq='W')[23],pd.date_range('2020','2021', freq='W')[23].weekday()

(Timestamp('2020-06-14 00:00:00', freq='W-SUN'), 6)

In [448]:
pd.date_range('20200601','20200712', freq='W')[0],pd.date_range('20200601','20200712', freq='W').weekday

(Timestamp('2020-06-07 00:00:00', freq='W-SUN'),
 Int64Index([6, 6, 6, 6, 6, 6], dtype='int64'))

In [442]:
pd.date_range('2020','2021', freq='W')

Timestamp('2020-06-14 00:00:00', freq='W-SUN')

## 三、重采样

#### 所谓重采样，就是指resample函数，它可以看做时序版本的groupby函数

### 1. resample对象的基本操作
#### 采样频率一般设置为上面提到的offset字符

In [462]:
df_r = pd.DataFrame(np.random.randint(0,10,3000).reshape(1000, 3),index=pd.date_range('20200101', freq='S', periods=1000),
                  columns=['A', 'B', 'C'])

In [463]:
df_r

,A,B,C
2020-01-01 00:00:00,0,8,4
2020-01-01 00:00:01,8,0,9
2020-01-01 00:00:02,9,5,9
2020-01-01 00:00:03,5,8,4
2020-01-01 00:00:04,5,5,2
...,...,...,...
2020-01-01 00:16:35,1,9,0
2020-01-01 00:16:36,6,7,8
2020-01-01 00:16:37,7,3,4
2020-01-01 00:16:38,0,7,7


In [464]:
r = df_r.resample('3min')
r

In [465]:
r.sum()

,A,B,C
2020-01-01 00:00:00,790,797,809
2020-01-01 00:03:00,767,783,870
2020-01-01 00:06:00,843,804,788
2020-01-01 00:09:00,840,798,830
2020-01-01 00:12:00,790,821,830
2020-01-01 00:15:00,436,480,415


In [467]:
df_r2 = pd.DataFrame(np.random.randint(0,10,600).reshape(200, 3),index=pd.date_range(start = '20200101', freq='D', periods=200), # 第一个参数为 start 
                  columns=['A', 'B', 'C'])
r = df_r2.resample('CBMS') # CBMS = custom business month start frequency
r.sum()

,A,B,C
2020-01-01,132,163,157
2020-02-03,138,132,129
2020-03-02,103,147,124
2020-04-01,126,110,104
2020-05-01,158,105,156
2020-06-01,121,112,140
2020-07-01,92,71,83


### 2. 采样聚合

In [468]:
r = df_r.resample('3T') # 类似于分组对象

In [474]:
# 和分组对象一样,支持迭代
for timestamp,df in r:
    print(timestamp)
    display(df.head(2))

2020-01-01 00:00:00


,A,B,C
2020-01-01 00:00:00,0,8,4
2020-01-01 00:00:01,8,0,9


2020-01-01 00:03:00


,A,B,C
2020-01-01 00:03:00,0,5,8
2020-01-01 00:03:01,0,8,5


2020-01-01 00:06:00


...

,A,B,C
2020-01-01 00:09:00,6,1,2
2020-01-01 00:09:01,6,4,8


2020-01-01 00:12:00


,A,B,C
2020-01-01 00:12:00,6,0,6
2020-01-01 00:12:01,3,8,8


2020-01-01 00:15:00


,A,B,C
2020-01-01 00:15:00,3,4,0
2020-01-01 00:15:01,1,7,1


In [475]:
r['A'].mean()

2020-01-01 00:00:00    4.388889
2020-01-01 00:03:00    4.261111
2020-01-01 00:06:00    4.683333
2020-01-01 00:09:00    4.666667
2020-01-01 00:12:00    4.388889
2020-01-01 00:15:00    4.360000
Freq: 3T, Name: A, dtype: float64

In [476]:
# 使用 agg 传入多个聚合函数
r['A'].agg([np.sum, np.mean, np.std])

,sum,mean,std
2020-01-01 00:00:00,790,4.388889,2.910948
2020-01-01 00:03:00,767,4.261111,2.930029
2020-01-01 00:06:00,843,4.683333,2.945454
2020-01-01 00:09:00,840,4.666667,2.955914
2020-01-01 00:12:00,790,4.388889,2.941494
2020-01-01 00:15:00,436,4.360000,2.955973


#### 类似地，可以使用函数/lambda表达式

In [477]:
r.agg({'A': np.sum,'B': lambda x: max(x)-min(x)})

,A,B
2020-01-01 00:00:00,790,9
2020-01-01 00:03:00,767,9
2020-01-01 00:06:00,843,9
2020-01-01 00:09:00,840,9
2020-01-01 00:12:00,790,9
2020-01-01 00:15:00,436,9


In [478]:
# 还可以尝试使用分组对象的其他的更多操作

### 3. 采样组的迭代
#### 采样组的迭代和groupby迭代完全类似，对于每一个组都可以分别做相应操作

In [479]:
small = pd.Series(range(6),index=pd.to_datetime(['2020-01-01 00:00:00', '2020-01-01 00:30:00'
                                                ,'2020-01-01 00:31:00', '2020-01-01 01:00:00'
                                                ,'2020-01-01 03:00:00', '2020-01-01 03:05:00']))
resampled = small.resample('H')
for name, group in resampled:
    print("Group: ", name)
    print("-" * 27)
    print(group, end="\n\n")

Group:  2020-01-01 00:00:00
---------------------------
2020-01-01 00:00:00    0
2020-01-01 00:30:00    1
2020-01-01 00:31:00    2
dtype: int64

Group:  2020-01-01 01:00:00
---------------------------
2020-01-01 01:00:00    3
dtype: int64

Group:  2020-01-01 02:00:00
---------------------------
Series([], dtype: int64)

Group:  2020-01-01 03:00:00
---------------------------
2020-01-01 03:00:00    4
2020-01-01 03:05:00    5
dtype: int64



## 四、窗口函数

#### 下面主要介绍pandas中两类主要的窗口(window)函数:rolling/expanding

In [480]:
s = pd.Series(np.random.randn(1000),index=pd.date_range('1/1/2020', periods=1000))
s.head()

2020-01-01    0.906773
2020-01-02   -0.131792
2020-01-03   -0.720544
2020-01-04   -0.167490
2020-01-05    0.042976
Freq: D, dtype: float64

### 1. Rolling
#### （a）常用聚合
#### 所谓rolling方法，就是规定一个窗口，它和groupby对象一样，本身不会进行操作，需要配合聚合函数才能计算结果

In [481]:
s.rolling(window=50)

Rolling [window=50,center=False,axis=0]

In [482]:
s.rolling??

Signature:
s.rolling(
    window,
    min_periods=None,
    center=False,
    win_type=None,
    on=None,
    axis=0,
    closed=None,
)
Docstring:
Provide rolling window calculations.

Parameters
----------
window : int, offset, or BaseIndexer subclass
    Size of the moving window. This is the number of observations used for
    calculating the statistic. Each window will be a fixed size.

    If its an offset then this will be the time period of each window. Each
    window will be a variable sized based on the observations included in
    the time-period. This is only valid for datetimelike indexes.

    If a BaseIndexer subclass is passed, calculates the window boundaries
    based on the defined ``get_window_bounds`` method. Additional rolling
    keyword arguments, namely `min_periods`, `center`, and
    `closed` will be passed to `get_window_bounds`.
min_periods : int, default None
    Minimum number of observations in window required to have a value
    (otherwise result is NA

In [483]:
s.rolling(window=50).mean()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04         NaN
2020-01-05         NaN
                ...   
2022-09-22    0.070524
2022-09-23    0.066079
2022-09-24    0.044344
2022-09-25    0.046146
2022-09-26    0.048335
Freq: D, Length: 1000, dtype: float64

In [484]:
s.rolling(window=4).mean() # 窗口为 4 , 则从第四行开始不为空

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04   -0.028263
2020-01-05   -0.244212
                ...   
2022-09-22   -0.263459
2022-09-23   -0.126175
2022-09-24   -0.304939
2022-09-25   -0.086165
2022-09-26   -0.311661
Freq: D, Length: 1000, dtype: float64

In [ ]:
# 注意窗口并不是分组, 而是用长度为窗口的框去套原来的数据,然后用窗口内的聚合值替代窗口最后一样的值.
# 因此窗口函数作用之后,pandas对象和原来具有相同的长度,但因为窗口的长度的原因,会在开头的window-1行引入缺失值

#### min_periods参数是指需要的非缺失数据点数量阀值

In [486]:
s.rolling(window=50,min_periods=3).mean().head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03    0.018146
2020-01-04   -0.028263
2020-01-05   -0.014015
Freq: D, dtype: float64

In [488]:
s.rolling(window=4).mean().head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04   -0.028263
2020-01-05   -0.244212
Freq: D, dtype: float64

In [489]:
# 在min_periods和window之间的行,也会有值,但窗口不是window,而是截止到该行的实际行数
s.rolling(window=4,min_periods=2).mean().head()

2020-01-01         NaN
2020-01-02    0.387491
2020-01-03    0.018146
2020-01-04   -0.028263
2020-01-05   -0.244212
Freq: D, dtype: float64

In [490]:
s.head()

2020-01-01    0.906773
2020-01-02   -0.131792
2020-01-03   -0.720544
2020-01-04   -0.167490
2020-01-05    0.042976
Freq: D, dtype: float64

In [491]:
# 特别地, 当min_periods = 1 时, 第一行会保留原始值,后边的window-2行会使用截止到该行的所有行作为窗口
s.rolling(window=4,min_periods=1).mean().head()

2020-01-01    0.906773
2020-01-02    0.387491
2020-01-03    0.018146
2020-01-04   -0.028263
2020-01-05   -0.244212
Freq: D, dtype: float64

#### count/sum/mean/median/min/max/std/var/skew/kurt/quantile/cov/corr都是常用的聚合函数
#### （b）rolling的apply聚合
#### 使用apply聚合时，只需记住传入的是window大小的Series，输出的必须是标量即可，比如如下计算变异系数

In [492]:
s.rolling(window=50,min_periods=3).apply(lambda x:x.std()/x.mean()).head()

2020-01-01          NaN
2020-01-02          NaN
2020-01-03    45.407273
2020-01-04   -24.028796
2020-01-05   -42.026062
Freq: D, dtype: float64

#### （c）基于时间的rolling

In [494]:
s.rolling('3D').mean().head(10)

2020-01-01    0.906773
2020-01-02    0.387491
2020-01-03    0.018146
2020-01-04   -0.339942
2020-01-05   -0.281686
2020-01-06    0.674334
2020-01-07    1.209815
2020-01-08    1.525169
2020-01-09    0.913665
2020-01-10    0.385628
Freq: D, dtype: float64

In [495]:
s.head(10)

2020-01-01    0.906773
2020-01-02   -0.131792
2020-01-03   -0.720544
2020-01-04   -0.167490
2020-01-05    0.042976
2020-01-06    2.147515
2020-01-07    1.438952
2020-01-08    0.989038
2020-01-09    0.313006
2020-01-10   -0.145162
Freq: D, dtype: float64

#### 可选closed='right'（默认）\'left'\'both'\'neither'参数，决定端点的包含情况

In [496]:
s.rolling('15D', closed='right').sum().head()
# 和之前学过的区间一致, 必然是一端开另一端闭

2020-01-01    0.906773
2020-01-02    0.774982
2020-01-03    0.054438
2020-01-04   -0.113053
2020-01-05   -0.070076
Freq: D, dtype: float64

### 2. Expanding

#### （a）expanding函数
#### 普通的expanding函数等价于rolling(window=len(s),min_periods=1)，是对序列的累计计算

In [497]:
s.rolling(window=len(s),min_periods=1).sum().head()

2020-01-01    0.906773
2020-01-02    0.774982
2020-01-03    0.054438
2020-01-04   -0.113053
2020-01-05   -0.070076
Freq: D, dtype: float64

In [498]:
s.expanding().sum().head()

2020-01-01    0.906773
2020-01-02    0.774982
2020-01-03    0.054438
2020-01-04   -0.113053
2020-01-05   -0.070076
Freq: D, dtype: float64

#### apply方法也是同样可用的

In [499]:
s.expanding().apply(lambda x:sum(x)).head()

2020-01-01    0.906773
2020-01-02    0.774982
2020-01-03    0.054438
2020-01-04   -0.113053
2020-01-05   -0.070076
Freq: D, dtype: float64

#### （b）几个特别的Expanding类型函数
#### cumsum/cumprod/cummax/cummin都是特殊expanding累计计算方法

In [500]:
s.cumsum().head()

2020-01-01    0.906773
2020-01-02    0.774982
2020-01-03    0.054438
2020-01-04   -0.113053
2020-01-05   -0.070076
Freq: D, dtype: float64

In [501]:
s.cumsum().head()

2020-01-01    0.906773
2020-01-02    0.774982
2020-01-03    0.054438
2020-01-04   -0.113053
2020-01-05   -0.070076
Freq: D, dtype: float64

#### shift/diff/pct_change都是涉及到了元素关系
#### ①shift是指序列索引不变，但值向后移动
#### ②diff是指前后元素的差，period参数表示间隔，默认为1，并且可以为负
#### ③pct_change是值前后元素的变化百分比，period参数与diff类似

In [502]:
s.shift(2).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03    0.906773
2020-01-04   -0.131792
2020-01-05   -0.720544
Freq: D, dtype: float64

In [503]:
s.diff(3).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04   -1.074263
2020-01-05    0.174768
Freq: D, dtype: float64

In [504]:
s.pct_change(3).head()

2020-01-01         NaN
2020-01-02         NaN
2020-01-03         NaN
2020-01-04   -1.184710
2020-01-05   -1.326093
Freq: D, dtype: float64

## 五、问题与练习

#### 【问题一】 如何对date_range进行批量加帧操作或对某一时间段加大时间戳密度？


#### 【问题二】 如何批量增加TimeStamp的精度？


#### 【问题三】 对于超出处理时间的时间点，是否真的完全没有处理方法？


#### 【问题四】 给定一组非连续的日期，怎么快速找出位于其最大日期和最小日期之间，且没有出现在该组日期中的日期？


#### 【练习一】 现有一份关于某超市牛奶销售额的时间序列数据，请完成下列问题：
#### （a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）

In [506]:
df=pd.read_csv('data/time_series_one.csv')
df.head()

,日期,销售额
0,2017/2/17,2154
1,2017/2/18,2095
2,2017/2/19,3459
3,2017/2/20,2198
4,2017/2/21,2413


In [517]:
df['dayofweek'] = df['日期'].astype('datetime64[ns]').dt.dayofweek

In [523]:
df.iloc[df['销售额'].idxmax(),:]

日期           2019/9/1
销售额              4333
dayofweek           6
Name: 926, dtype: object

#### （b）计算除去春节、国庆、五一节假日的月度销售总额

#### （c）按季度计算周末（周六和周日）的销量总额

#### （d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和

#### （e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）

#### 【练习二】 继续使用上一题的数据，请完成下列问题：
#### （a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）

In [534]:
df2a= df[['日期','销售额']].set_index('日期').rolling(window=50,min_periods=1).agg([np.mean,np.max]).join(df[['日期','销售额']].set_index('日期'))
df2a
# UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(销售额, mean)","(销售额, amax)",销售额
日期,,,
2017/2/17,2154.000000,2154.0,2154
2017/2/18,2124.500000,2154.0,2095
2017/2/19,2569.333333,3459.0,3459
2017/2/20,2476.500000,3459.0,2198
2017/2/21,2463.800000,3459.0,2413
...,...,...,...
2019/11/9,2955.860000,4048.0,3022
2019/11/10,2962.060000,4048.0,2961
2019/11/11,2988.780000,4048.0,3984


#### （b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果

In [544]:
df2b=df[['日期','销售额']].set_index('日期').rolling(window=5,min_periods=1).agg([np.mean,np.max]).droplevel(0,axis=1).join(df[['日期','销售额']].set_index('日期'))
df2b

,mean,amax,销售额
日期,,,
2017/2/17,2154.000000,2154.0,2154
2017/2/18,2124.500000,2154.0,2095
2017/2/19,2569.333333,3459.0,3459
2017/2/20,2476.500000,3459.0,2198
2017/2/21,2463.800000,3459.0,2413
...,...,...,...
2019/11/9,2860.600000,3133.0,3022
2019/11/10,2939.000000,3133.0,2961
2019/11/11,3213.800000,3984.0,3984


In [548]:
df2b['flag']=(df2b['销售额']>df2b['mean']).astype('int64')
df2b

,mean,amax,销售额,flag
日期,,,,
2017/2/17,2154.000000,2154.0,2154,0
2017/2/18,2124.500000,2154.0,2095,0
2017/2/19,2569.333333,3459.0,3459,1
2017/2/20,2476.500000,3459.0,2198,0
2017/2/21,2463.800000,3459.0,2413,0
...,...,...,...,...
2019/11/9,2860.600000,3133.0,3022,1
2019/11/10,2939.000000,3133.0,2961,1
2019/11/11,3213.800000,3984.0,3984,1


#### （c）将(b)中的“向前5天”改为“向前非周末5天”，请再次计算结果